In [ ]:
from lxml import etree
import glob

In [ ]:
from pathlib import Path
from tqdm import tqdm

In [ ]:
import pandas as pd
import numpy as np
import os,sys

In [ ]:
import re
from PIL import Image

### 1.Data Loader.

In [13]:
root = Path('/nas/chenyi/datasets_nas/dada_det/xml/sys_hard_sample_xml/hard_sample_xml_0610/')

In [14]:
xml_list = glob.glob(str(root /'*.xml'))
# xml_list += glob.glob(str(root /'*/*/*.xml'))
# xml_list += glob.glob(str(root /'*/*/*/*.xml'))
len(xml_list)

60

In [15]:
# 核查xml是否有重复
xml_df = pd.DataFrame()
xml_df['path'] = xml_list
xml_df['stem'] = xml_df['path'].apply(lambda x: Path(x).stem)
len(xml_df),len(xml_df.drop_duplicates('stem'))

(60, 60)

### 接口

In [16]:
def voc2yolo():
    center_x = (xmin + (xmax-xmin)/2)/width
    center_y = (ymin + (ymax-ymin)/2)/height
    bdb_w = (xmax-xmin)/width
    bdb_h = (ymax-ymin)/height

In [17]:
def parse_xml(p):
    trees = etree.parse(p)
    width, height = int(trees.find('size/width').text), int(trees.find('size/height').text)
    bboxes = []
    for obj in trees.findall('object'):
        bx = []
        bx.append(p)
        bx.append(int(width))
        bx.append(int(height))
        bdb = obj.find('bndbox')
        bx.append(obj.find('name').text)
        bx.append(int(bdb.find('xmin').text))
        bx.append(int(bdb.find('ymin').text))
        bx.append(int(bdb.find('xmax').text))
        bx.append(int(bdb.find('ymax').text))
        bboxes.append(bx)
    return bboxes

In [97]:
model_label_map = {
    0: 10,  # 鞋子
    1: 8,  # 包包
    2: 4,  # 上装
    3: 5,  # 裤子
    4: 6,  # 裙子
    5: 7,  # 连体装
}

### 解析XML

In [18]:
bboxes_list = []
for p in tqdm(xml_list):
    row = parse_xml(p)
    bboxes_list += row
    # break

100%|████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 228.01it/s]


In [19]:
df = pd.DataFrame(bboxes_list)

In [20]:
df.head(2).values

array([['/nas/chenyi/datasets_nas/dada_det/xml/sys_hard_sample_xml/hard_sample_xml_0610/0bd04a4b-cef0-48d6-8fd2-d5a399012059.xml',
        3024, 4032, '连衣裙', 170, 220, 2588, 3890],
       ['/nas/chenyi/datasets_nas/dada_det/xml/sys_hard_sample_xml/hard_sample_xml_0610/lQDPDht0B1atyB7NBwDNBUCw0OcCfSxkGL8CjsZJ5kDJAA_1344_1792.jpg_720x720g.xml',
        540, 720, '连衣裙', 170, 78, 461, 709]], dtype=object)

In [22]:
# df['man'] = df[int(df.columns[0])].apply(lambda x: re.findall('wholebody0609_xml/.*?_(.*?)/', x)[0])

In [ ]:
df.value_counts('man')

In [137]:
# df.to_csv('../data/sys_error_sample/sys_error_june.csv', index=False)

### 数据展示

In [24]:
!tree -d -L 1 /home/chenyi/workspace/dataset/data/dada_det

/home/chenyi/workspace/dataset/data/dada_det
├── hard_sample_0610
├── __MACOSX
├── shoe_bag11k_may
├── shoeBag5k0515
└── wholebody0601_all

5 directories


In [25]:
ROOT = Path('/home/chenyi/workspace/dataset/data/dada_det/hard_sample_0610/')

#### 获取对应的图像路径

In [ ]:
!tree -d -L 1 /home/chenyi/workspace/dataset/data/dada_det/

In [26]:
bboxes_df = pd.read_csv('../data/sys_error_sample/sys_error_june.csv')

In [27]:
bboxes_df['stem'] = bboxes_df[bboxes_df.columns[0]].apply(lambda x: Path(x).stem)

In [35]:
img_list = glob.glob(str(ROOT /'*.jpg'))

In [36]:
img_df = pd.DataFrame()
img_df['path'] = img_list

In [37]:
img_df['stem'] = img_df['path'].apply(lambda x: Path(x).stem)

In [38]:
# xml是否都有对应的图片
cross_df = bboxes_df[bboxes_df['stem'].isin(img_df['stem'])]
len(cross_df), len(bboxes_df), len(cross_df.drop_duplicates('stem')), len(bboxes_df.drop_duplicates('stem'))

(60, 60, 60, 60)

In [39]:
# 图片是否有重复
len(img_df), len(img_df.drop_duplicates('stem'))

(60, 60)

##### 获取bbox对应的图片

In [130]:
# 对应的图片
img_df = img_df[img_df['stem'].isin(bboxes_df['stem'])]

In [131]:
gp = dict(list(img_df.groupby('stem')))

In [132]:
stem2path = {}
for path,stem in zip(img_df['path'].values, img_df['stem'].values):
    stem2path[stem] = path

In [133]:
df = bboxes_df
p_list = []
for ind, row in df.iterrows():
    stem = row['stem']
    p = stem2path[stem]
    p_list.append(p)

In [134]:
stem2path[stem]

'/home/chenyi/workspace/dataset/data/dada_det/hard_sample_0610/3da34321-75d6-4415-8fd3-a88b83bf9dd8.jpg'

In [135]:
df['img_sp'] = p_list

##### 获取图片的宽和高

In [138]:
w_list = []
h_list = []
for ind,row in tqdm(df.iterrows()):
    img_sp = row['img_sp']
    img = Image.open(img_sp)
    w, h = img.size
    w_list.append(w)
    h_list.append(h)
    # break

60it [00:00, 879.36it/s]


In [139]:
df[df.columns[1]] = w_list
df[df.columns[2]] = h_list

In [140]:
df.head(2)

,0,1,2,3,4,5,6,7,stem,label_model,label6,img_sp
0,/nas/chenyi/datasets_nas/dada_det/xml/sys_hard_sample_xml/hard_sample_xml_0610/0bd04a4b-cef0-48d6-8fd2-d5a399012059.xml,3024,4032,连衣裙,170,220,2588,3890,0bd04a4b-cef0-48d6-8fd2-d5a399012059,5,7,/home/chenyi/workspace/dataset/data/dada_det/hard_sample_0610/0bd04a4b-cef0-48d6-8fd2-d5a399012059.jpg
1,/nas/chenyi/datasets_nas/dada_det/xml/sys_hard_sample_xml/hard_sample_xml_0610/lQDPDht0B1atyB7NBwDNBUCw0OcCfSxkGL8CjsZJ5kDJAA_1344_1792.jpg_720x720g.xml,540,720,连衣裙,170,78,461,709,lQDPDht0B1atyB7NBwDNBUCw0OcCfSxkGL8CjsZJ5kDJAA_1344_1792.jpg_720x720g,5,7,/home/chenyi/workspace/dataset/data/dada_det/hard_sample_0610/lQDPDht0B1atyB7NBwDNBUCw0OcCfSxkGL8CjsZJ5kDJAA_1344_1792.jpg_720x720g.jpg


In [141]:
# df.to_csv('../data/sys_error_sample/sys_error_june.csv', index=False)

#### 处理打标标签

##### 构建中文标签和业务id映射

In [45]:
# counter = df.value_counts(df.columns[3])
# name_list = [item[0] for item in list(counter.index)]
# name_df = pd.DataFrame()
# name_df['name'] = name_list

In [121]:
import json
with open('../../../data/dict/name_chi2l1.json', 'r') as f:
    name_chi2l1 = json.load(f)

In [55]:
def get_list2dict(k_list, v_list):
    d = {}
    for item in zip(k_list, v_list):
        d[item[0]] = item[1]
    return d

In [122]:
df['label6'] = df[df.columns[3]].map(name_chi2l1)

In [123]:
df.value_counts('label6')

label6
4    27
7    19
6    8 
8    4 
5    2 
dtype: int64

In [115]:
# 增加多出的中文标签映射
# name_chi2l1['皮衣外套'] = 4

In [116]:
import json
with open('../../../data/dict/name_chi2l1.json', 'w') as f:
    json.dump(name_chi2l1, f)

In [124]:
model_label_map

{0: 10, 1: 8, 2: 4, 3: 5, 4: 6, 5: 7}

In [126]:
l1_map_model = get_list2dict(model_label_map.values(), model_label_map.keys())
df['label_model'] = df['label6'].map(l1_map_model)

##### 存储目标index

In [ ]:
# df = pd.read_csv('../data/wh.csv')

In [ ]:
!ls /nas/chenyi/datasets_cls/dada_det_drawed/wholebody_jul_drawed

In [ ]:
!mkdir -p /nas/chenyi/datasets_cls/dada_det_drawed/wholebody_jul_drawed

In [ ]:
draw_root = Path('/nas/chenyi/datasets_cls/dada_det_drawed/wholebody_jul_drawed')

In [ ]:
df.head(2).values

In [ ]:
df['img_tp'] = df['img_sp'].apply(lambda x:'{0}/{1}/{2}'.format(draw_root, Path(x).parts[-2], Path(x).name))

In [ ]:
sys.path.append('../../../')
from utils.plots.plots import draw_bbox
import cv2
from PIL import Image

In [ ]:
# error_p_list = []
# for ind, row in tqdm(df.iterrows()):
#     row_info = row.loc[df.columns[:8]].values
#     w,h = row_info[1], row_info[2]
#     name = row.loc[['label_model']].values[0]
#     bbox = list(row_info[4:8])
#     bbox.insert(0,name)
#     tp = Path(row.loc[['img_tp']].values[0])
#     if not tp.parent.is_dir():
#         tp.parent.mkdir()
#     rp = tp if tp.is_file() else row.loc[['img_sp']].values[0]
#     img = cv2.imread(str(rp))
#     img = draw_bbox(img, bbox)
#     try:
#         cv2.imwrite(str(tp), img)
#     except:
#         error_p_list.append(tp)
#     # break

In [ ]:
error_p_list

In [ ]:
error_df = pd.DataFrame()
error_df['path'] = error_p_list
error_df['stem'] = error_df['path'].apply(lambda x: Path(x).stem)

In [ ]:
df.head()

In [ ]:
df = df[~df['stem'].isin(error_df['stem'])]

In [ ]:
# df.to_csv('../data/wholebody5k0610/wholebody5k_july.csv', index=False)

In [ ]:
Image.fromarray(img)

### Display Html

In [ ]:
sys.path.append('../../../')
from utils.display.html import *
from IPython.display import HTML
from io import BytesIO

In [ ]:
draw_root = Path('/nas/chenyi/datasets_cls/dada_det_drawed/wholebody_jul_drawed/')

In [ ]:
img_list = glob.glob(str(draw_root/'*/*.jpg'))
len(img_list), img_list[-1]

In [ ]:
# img_list = glob.glob(draw_root + '*.jpg')
df_drawed = pd.DataFrame()
df_drawed['path'] = img_list[-100:]

In [ ]:
size = 500
df_drawed['res'] = df_drawed['path'].map(lambda x: get_thumbnail(x, size))

In [ ]:
f = df_drawed[['res', 'path']].to_html(formatters={'res': image_formatter}, escape=False)

In [ ]:
HTML(f)

### 解析json文件

In [ ]:
json_list = glob.glob('/home/chenyi/workspace/dataset/haowei34k_labelImg/xml_label/*/*.json')

In [ ]:
json_df = pd.DataFrame()
json_df['spath'] = json_list

In [ ]:
xml_dir = Path('/home/chenyi/workspace/dataset/haowei34k_labelImg/json2xml/')

In [ ]:
json_df['tpath'] = json_df['spath'].apply(lambda x:
                                         '{0}/{1}'.format(xml_dir, re.findall(r'/[0-9]{1,3}/(.*?.json)',x)[0]))

In [ ]:
sys.path.append('/home/chenyi/workspace/myPyModule99/data_format/json_to_xml')
from json_to_xml.main import *

In [ ]:
root_json_dir = xml_dir
root_save_xml_dir = '/home/chenyi/workspace/dataset/haowei34k_labelImg/json2xml/xml'

In [ ]:
import json
for json_filename in tqdm(os.listdir(root_json_dir)):
    json_path = os.path.join(root_json_dir, json_filename)
    json_data = json.load(open(json_path))
    break

In [ ]:
json_df['path'] = glob.glob(str(root_json_dir/'*.json'))

In [ ]:
json_df['fname'] =  json_df['path'].apply(lambda x: re.findall(r'/json2xml/(.*?).json',x)[0])

In [ ]:
json_labels = {}
for ind,row in tqdm(json_df.iterrows()):
    json_path = row['path']
    fname = row['fname']
    json_data = json.load(open(json_path))
    json_labels[fname] = ''
    for item in json_data['shapes']:
        name = item['label']
        points = item['points']
        xmin, ymin = points[0]
        xmax, ymax = points[1]
        
        if name not in classes_name.keys():
            classes_name[name] = len(classes_name)
        json_labels[fname] += '{} {:6f} {:6f} {:6f} {:6f}\n'.format(
            classes_name[name], xmin, ymin, xmax, ymax)

In [ ]:
fname_list = []
xmin_list = []
ymin_list = []
xmax_list = []
ymax_list = []
labels_list = []
for ind,row in tqdm(json_df.iterrows()):
    json_path = row['path']
    fname = row['fname']
    json_data = json.load(open(json_path))
    for item in json_data['shapes']:
        name = item['label']
        points = item['points']
        xmin, ymin = points[0]
        xmax, ymax = points[1]
        
        fname_list.append(fname)
        labels_list.append(name)
        xmin_list.append(xmin)
        ymin_list.append(ymin)
        ymax_list.append(ymax)
        xmax_list.append(xmax)

In [ ]:
json_bboxes = pd.DataFrame()

In [ ]:
json_bboxes['fname'] = fname_list
json_bboxes['label'] = labels_list
json_bboxes['xmin'] = xmin_list
json_bboxes['ymin'] = ymin_list
json_bboxes['xmax'] = xmax_list
json_bboxes['ymax'] = ymax_list

In [ ]:
json_bboxes.head()

### pandas处理

In [ ]:
labels_p

In [ ]:
xml_list = glob.glob(str(labels_p /'*.xml'))
xmlFiles = [re.findall(r'/xml/(.*?.xml)',x)[0] for x in xml_list]
fname_list = []
xmin_list = []
ymin_list = []
xmax_list = []
ymax_list = []
labels_list = []
for ind,xmlFile in tqdm(enumerate(xmlFiles)):
    trees = etree.parse(labels_p /xmlFile)
    width, height = int(trees.find('size/width').text), int(trees.find('size/height').text)
    # fname = re.findall(r'[0-9]{1,3}/(.*?).xml',xmlFile)[0]     
    fname = xmlFile[:-4]     

    for obj in trees.findall('object'):
        bdb = obj.find('bndbox')
        name = obj.find('name').text
        xmin = int(bdb.find('xmin').text)
        ymin = int(bdb.find('ymin').text)
        xmax = int(bdb.find('xmax').text)
        ymax = int(bdb.find('ymax').text)
        
        fname_list.append(fname)
        labels_list.append(name)
        xmin_list.append(xmin)
        ymin_list.append(ymin)
        ymax_list.append(ymax)
        xmax_list.append(xmax)

In [ ]:
xml_bboxes = pd.DataFrame()
xml_bboxes['fname'] = fname_list
xml_bboxes['label'] = labels_list
xml_bboxes['xmin'] = xmin_list
xml_bboxes['ymin'] = ymin_list
xml_bboxes['xmax'] = xmax_list
xml_bboxes['ymax'] = ymax_list

In [ ]:
xml_bboxes.head()

In [ ]:
xml_bboxes.info()

In [ ]:
labelme_df = xml_bboxes

In [ ]:
# labelme_df.to_csv('/home/chenyi/workspace/dataset/haowei34k_labelImg/labelme_hw34k.csv', index=False)

In [ ]:
labelme_df['label'].value_counts().axes,len(labelme_df['label'].value_counts().axes[0])

In [ ]:
import pickle
with open('labelme_names_dict.pickle', 'rb') as f:
    names_dict = pickle.load(f)

In [ ]:
# import pickle
# # with open('labelme_names_dict.pickle', 'wb') as f:
#     pickle.dump(names_dict, f)

In [ ]:
labelme_df['label43'] = labelme_df['label'].apply(lambda x: names_dict[x])

In [ ]:
labelme_df.to_csv('labelme_voc_bboxes_relabel.csv', index=False)

In [ ]:
labelme_df['label43'].value_counts()

In [ ]:
root = Path('/home/chenyi/workspace/dataset/haowei34k/')

In [ ]:
imgs_list = glob.glob(str(root /'images/*/*.jpg'))

In [ ]:
imgs_df = pd.DataFrame()
imgs_df['path'] = imgs_list

In [ ]:
import re

In [ ]:
imgs_df['fname']= imgs_df['path'].apply(lambda x: re.findall(r'images/.*?/(.*?).jpg', x)[0])

In [ ]:
imgs_df.head(2).values

In [ ]:
_ = [os.symlink(p, p.replace()) for p in imgs_list]

In [ ]:
labelme_df['fname'] = labelme_df['fname'].astype(str)

In [ ]:
imgs_df.info()

In [ ]:
labelme_df.drop_duplicates('fname').info()

In [ ]:
x = labelme_df[~labelme_df['fname'].isin(imgs_df['fname'])]

In [ ]:
y = imgs_df[~imgs_df['fname'].isin(labelme_df['fname'])]

In [ ]:
y.info()

In [ ]:
labelme_df.head()

In [ ]:
root

In [ ]:
from PIL import Image

In [ ]:
w_list = []
h_list = []
for ind, row in tqdm(imgs_df.iterrows()):
    p_ = row['path']
    try:
        img = Image.open(str(p_))
        w,h = img.size
    except:
        w,h = 0,0
        print(p_)
    w_list.append(w)
    h_list.append(h)  

In [ ]:
imgs_df['width'] = w_list
imgs_df['height'] = h_list

In [ ]:
imgs_df['mode'] = imgs_df['path'].apply(lambda x: re.findall(r'/images/([a-z]{3,5})/', x)[0])

In [ ]:
val_df = imgs_df[imgs_df['mode']=='val']

In [ ]:
labelme_df['mode'] = 'train'

In [ ]:
labelme_df.loc[labelme_df[labelme_df['fname'].isin(val_df['fname'])].index,'mode'] = 'val'

In [ ]:
labelme_df['width'] = w_list
labelme_df['height'] = h_list

In [ ]:
names_lab2num = {}
for k,v in names.items():
    names_lab2num[v[0]] = k

In [ ]:
labelme_df.to_csv('haowei34k_voc_bboxes_relabel.csv', index=False)

In [ ]:
labelme_df['label_num'] = labelme_df['label43'].apply(lambda x:
                                                     names_lab2num[x])